In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from brick.azr import AZR
from lmfit import Parameters, Minimizer, report_fit, conf_interval

from tqdm import tqdm

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Nuisance parameter map : { index : (value, error) } (in AZURE2 order)
nuisances = { 0 : (1.63, 0.12), 
             13: (1, 0.05), 14: (1, 0.05),  15: (1, 0.05),  
             16: (1, 0.069),  17: (1, 0.079),  18: (1, 0.1),  
             19: (1, 0.1),  20: (1, 0.1), 21: (1, 0.1),  
             22: (1, 0.1),  23: (1, 0.0), 24: (1, 0.0), 
             25: (1, 0.0), 26: (1, 0.0), 27: (1, 0.0) }

In [2]:
# Convert the covariance matrix into a correlation matrix
def corr_from_cov(cov):
    v = np.sqrt(np.diag(cov))
    outer_v = np.outer(v, v)
    corr = cov / outer_v
    corr[cov == 0] = 0
    return corr

In [3]:
# We read the .azr file and set the external capture file to speed up the calculation
azr = AZR('12c_pg.azr')
azr.ext_capture_file ='output/intEC.dat'

# We get the initial values from AZURE2
theta0  = azr.config.get_input_values()
ntheta  = len(theta0)
nparams = len(azr.config.parameters)

In [4]:
# We'll read the data from the output file since it's already in the center-of-mass frame
data    = np.vstack([np.loadtxt('output/'+f) for f in azr.config.data.output_files])
x       = data[:, 0]
y       = data[:, 5]
dy_bare = data[:, 6]

In [5]:
# Callback function to print the chi2 at each iteration
def callback(params, iter, resid, *args, **kws):
    if( iter % 10 == 0 ): print("Iteration: {:6d} it Chi2: {:15.4f}".format( iter, resid ), end="\r" )

# Add nuisance parameters to the fit
def nuisance(theta):
    chi2 = 0
    for i in nuisances:
        if( nuisances[i][1] == 0 ): continue
        else: chi2 += pow((theta[i] - nuisances[i][0])/nuisances[i][1], 2)
    return chi2

# Function to minimize
def fcn(theta):
    theta = list( theta.valuesdict().values() )
    output = np.vstack(azr.predict(theta, dress_up=False))
    mu, y, dy = output[:, 3], output[:, 5], output[:, 6]
    return np.sum( pow( (y - mu) / dy, 2 ) ) + nuisance(theta)

In [6]:
params = Parameters()
for i in range(ntheta):
    if( theta0[i] > 0 ): params.add( "param_{}".format(i), value=theta0[i], vary=True, min=0, max=theta0[i] * 100 )
    else: params.add( "param_{}".format(i), value=theta0[i], vary=True, min=theta0[i] * 100, max=0 )

mini = Minimizer( fcn, params, iter_cb=callback )

In [7]:
# Minimization
out = mini.minimize(method="lbfgsb")

In [8]:
theta, error = [], []
print('Parameter    Value       Stderr')
for name, param in out.params.items():
    if( param.stderr == None ): param.stderr = 0
    theta.append( param.value )
    error.append( param.stderr )
    print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')

np.savetxt( "lmfit/best_chi2.txt", np.transpose([theta,error]), fmt='%1.4e %1.4e' )

Parameter    Value       Stderr
param_0     1.74135     0.00000
param_1     2.36963     0.00000
param_2 36407.44750     0.00000
param_3    -0.49870     0.00000
param_4  -716.90444     0.00000
param_5     3.50500     0.00000
param_6 53825.54791     0.00000
param_7    -0.50086     0.00000
param_8     0.00224     0.00000
param_9  -481.76750     0.00000
param_10  3975.00153     0.00000
param_11     3.54701     0.00000
param_12 47380.24696     0.00000
param_13     1.01924     0.00000
param_14     1.01605     0.00000
param_15     1.00867     0.00000
param_16     1.09796     0.00000
param_17     1.09043     0.00000
param_18     1.15445     0.00000
param_19     0.93554     0.00000
param_20     1.11344     0.00000
param_21     0.86157     0.00000
param_22     0.83001     0.00000
param_23     1.48890     0.00000
param_24     1.27733     0.00000
param_25     1.17352     0.00000
param_26     1.18250     0.00000
param_27     1.09923     0.00000


In [9]:
# Minimization
out2 = mini.minimize(method="least-squares", params=out.params)

In [ ]:
theta, error = [], []
print('Parameter    Value       Stderr')
for name, param in out.params.items():
    if( param.stderr == None ): param.stderr = 0
    theta.append( param.value )
    error.append( param.stderr )
    print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')